In [1]:
import os

In [2]:
%pwd

'/Users/ronaksatpute/Projects/MLOps/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ronaksatpute/Projects/MLOps'

In [5]:
import pandas as pd
data = pd.read_csv("/Users/ronaksatpute/Projects/MLOps/artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path

In [7]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [9]:
import os
from src.MLProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)
        train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

        train_data_path = os.path.join(self.config.root_dir, "train.csv")
        test_data_path = os.path.join(self.config.root_dir, "test.csv")

        train_data.to_csv(train_data_path, index=False)
        test_data.to_csv(test_data_path, index=False)

        logger.info(f"Train and test data saved at {train_data_path} and {test_data_path} respectively.")
        logger.info(train_data.shape)
        logger.info(test_data.shape)

        print(train_data.shape)
        print(test_data.shape)

In [11]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2025-12-05 12:38:04,192:INFO:common:YAML file: configs/config.yaml loaded successfully]
[2025-12-05 12:38:04,193:INFO:common:YAML file: params.yaml loaded successfully]
[2025-12-05 12:38:04,194:INFO:common:YAML file: schema.yaml loaded successfully]
[2025-12-05 12:38:04,194:INFO:common:Directory created at: artifacts]
[2025-12-05 12:38:04,195:INFO:common:Directory created at: artifacts/data_transformation]
[2025-12-05 12:38:04,202:INFO:3549672999:Train and test data saved at artifacts/data_transformation/train.csv and artifacts/data_transformation/test.csv respectively.]
[2025-12-05 12:38:04,202:INFO:3549672999:(1279, 12)]
[2025-12-05 12:38:04,202:INFO:3549672999:(320, 12)]
(1279, 12)
(320, 12)
